# SMTP Email Sending - Complete Process Explanation

This notebook demonstrates two approaches to sending emails via SMTP (Simple Mail Transfer Protocol):
1. **High-level approach** using Python's `smtplib` library
2. **Low-level approach** using raw socket connections

## SMTP Protocol Overview

SMTP is a communication protocol used for sending emails. It operates on **port 587** (with STARTTLS) or **port 465** (with SSL/TLS). Gmail requires secure authentication.

## Process Flow

### 1. Connection Establishment
- Connect to Gmail's SMTP server (`smtp.gmail.com:587`)
- Server responds with a greeting message (220 status code)

### 2. EHLO Command
- Send `EHLO` (Extended Hello) to identify the client
- Server responds with supported extensions and capabilities

### 3. Security (STARTTLS)
- Send `STARTTLS` command to initiate TLS encryption
- Server confirms TLS support
- **Critical**: Wrap the socket with SSL/TLS encryption before continuing

### 4. Re-authentication with EHLO
- Send `EHLO` again after TLS is established
- Server lists capabilities available over secure connection

### 5. Authentication
- Send `AUTH LOGIN` to start authentication
- Server requests username (base64 encoded)
- Send base64-encoded email address
- Server requests password
- Send base64-encoded app password

### 6. Email Composition
- `MAIL FROM:` - Specify sender's email address
- `RCPT TO:` - Specify recipient's email address
- `DATA` - Begin email content transmission

### 7. Email Headers and Body
- Send email headers (From, To, Subject)
- Send empty line to separate headers from body
- Send email body content
- Send `.` (dot) on a new line to indicate end of message

### 8. Connection Termination
- Send `QUIT` command
- Server acknowledges and closes connection

## Key Differences Between Approaches

**Using `smtplib`**: Python handles all protocol details automatically
**Using raw sockets**: Manual implementation of each SMTP command and response handling

In [4]:
import smtplib

server = smtplib.SMTP('smtp.gmail.com', 587)
server.starttls()
server.login("vaultxcassiopeia@gmail.com", "mizi xvka gosk uvbz")
msg = """From: vaultxcassiopeia@gmail.com
To: aniskum59431@gmail.com
Subject: Lab mail

SMTP lab assignment...
"""
server.sendmail("vaultxcassiopeia@gmail.com", "aniskum59431@gmail.com", msg)
print("Email sent")
server.quit()



Email sent


(221,
 b'2.0.0 closing connection 98e67ed59e1d1-32b555b9d83sm18260553a91.17 - gsmtp')

In [ ]:
import base64
import socket
import ssl

# Gmail credentials
username = "vaultxcassiopeia@gmail.com"
password = "mizi xvka gosk uvbz"  # App password, not regular password

# SMTP requires base64 encoding for authentication
sender64 = base64.b64encode(username.encode()).decode()
pass64 = base64.b64encode(password.encode()).decode()

print("Username (base64):", sender64)
print("Password (base64):", pass64)

# === MANUAL SMTP IMPLEMENTATION ===
print("\n=== Starting Manual SMTP Process ===")

# Step 1: Create socket and connect to Gmail's SMTP server
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
s.connect(("smtp.gmail.com", 587))
response = s.recv(1024)
print("1. Server greeting:", response.decode().strip())

# Step 2: Send EHLO to identify ourselves
s.send(b"EHLO smtp.gmail.com\r\n")
response = s.recv(1024)
print("2. EHLO response:", response.decode().strip())

# Step 3: Request TLS encryption (required by Gmail)
s.send(b"STARTTLS\r\n")
response = s.recv(1024)
print("3. STARTTLS response:", response.decode().strip())

# Step 4: CRITICAL - Wrap socket with TLS encryption
print("4. Establishing TLS encryption...")
context = ssl.create_default_context()
s = context.wrap_socket(s, server_hostname="smtp.gmail.com")

# Step 5: Send EHLO again over encrypted connection
s.send(b"EHLO smtp.gmail.com\r\n")
response = s.recv(1024)
print("5. EHLO after TLS:", response.decode().strip())

# Step 6: Begin authentication process
s.send(b"AUTH LOGIN\r\n")
response = s.recv(1024)
print("6. AUTH LOGIN response:", response.decode().strip())

# Step 7: Send base64-encoded username
s.send(sender64.encode() + b"\r\n")
response = s.recv(1024)
print("7. Username response:", response.decode().strip())

# Step 8: Send base64-encoded password
s.send(pass64.encode() + b"\r\n")
response = s.recv(1024)
print("8. Password response:", response.decode().strip())

# Step 9: Specify sender email address
s.send(b"MAIL FROM:<vaultxcassiopeia@gmail.com>\r\n")
response = s.recv(1024)
print("9. MAIL FROM response:", response.decode().strip())

# Step 10: Specify recipient email address
s.send(b"RCPT TO:<aniskum59431@gmail.com>\r\n")
response = s.recv(1024)
print("10. RCPT TO response:", response.decode().strip())

# Step 11: Begin email data transmission
s.send(b"DATA\r\n")
response = s.recv(1024)
print("11. DATA response:", response.decode().strip())

# Step 12: Send email headers
print("12. Sending email headers and body...")
s.send(b"From:<vaultxcassiopeia@gmail.com>\r\n")
s.send(b"To:<aniskum59431@gmail.com>\r\n")
s.send(b"Subject: Test Email\r\n")
s.send(b"\r\n")  # Empty line separates headers from body

# Step 13: Send email body and end marker
s.send(b"Lab mail smtp computer network\r\n")
s.send(b".\r\n")  # Dot on new line indicates end of message
response = s.recv(1024)
print("13. Email sent response:", response.decode().strip())

# Step 14: Close the connection gracefully
s.send(b"QUIT\r\n")
response = s.recv(1024)
print("14. QUIT response:", response.decode().strip())

s.close()
print("15. Connection closed successfully")

Username (base64): dmF1bHR4Y2Fzc2lvcGVpYUBnbWFpbC5jb20=
Password (base64): bWl6aSB4dmthIGdvc2sgdXZieg==
Server greeting: 220 smtp.gmail.com ESMTP d2e1a72fcca58-774662cb574sm1439459b3a.87 - gsmtp

Server greeting: 220 smtp.gmail.com ESMTP d2e1a72fcca58-774662cb574sm1439459b3a.87 - gsmtp

EHLO response: 250-smtp.gmail.com at your service, [14.139.194.121]
250-SIZE 35882577
250-8BITMIME
250-STARTTLS
250-ENHANCEDSTATUSCODES
250-PIPELINING
250-CHUNKING
250 SMTPUTF8

EHLO response: 250-smtp.gmail.com at your service, [14.139.194.121]
250-SIZE 35882577
250-8BITMIME
250-STARTTLS
250-ENHANCEDSTATUSCODES
250-PIPELINING
250-CHUNKING
250 SMTPUTF8

STARTTLS response: 220 2.0.0 Ready to start TLS

STARTTLS response: 220 2.0.0 Ready to start TLS

EHLO after TLS response: 250-smtp.gmail.com at your service, [14.139.194.121]
250-SIZE 35882577
250-8BITMIME
250-AUTH LOGIN PLAIN XOAUTH2 PLAIN-CLIENTTOKEN OAUTHBEARER XOAUTH
250-ENHANCEDSTATUSCODES
250-PIPELINING
250-CHUNKING
250 SMTPUTF8

EHLO after TLS re